In [7]:
import folium
import pandas as pd
import pickle
import numpy as np

In [85]:
file = open('./pickles/house_latlong.p','rb')
df = pickle.load(file)

In [10]:
ave_lat = np.mean(df.latitude)
ave_long = np.mean(df.longitude)
(ave_lat, ave_long)

(42.03438790059312, -93.64255447330521)

In [69]:
m = folium.Map(location=[ave_lat, ave_long],tiles='Stamen Toner')

In [70]:
neighborhoods = df[~np.isnan(df.latitude)].groupby('Neighborhood')[['latitude','longitude']].mean() #get center of neighborhood
neighborhoods['num_houses'] = df.Neighborhood.value_counts()
nieghborhoods = neighborhoods.reset_index()

In [71]:
for i in range(0,neighborhoods.shape[0]):
    folium.Circle(location = [neighborhoods.latitude[i], neighborhoods.longitude[i]], radius = int(neighborhoods.num_houses[i]), popup = nieghborhoods.Neighborhood[i]).add_to(m)
    
    #color the circles

In [72]:
m

In [227]:
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.model_selection import train_test_split

In [268]:
KNR = KNeighborsRegressor(n_neighbors=50)

In [190]:
#dropping NA values with lat long. 
df = df[~np.isnan(df.latitude)]
#redo this with the cosine(42) multiplication. 

In [246]:
x_train, x_test, y_train, y_test = train_test_split(df.drop('SalePrice', axis = 1),df['SalePrice'], test_size = .5)

In [269]:
KNR.fit(x_train[['latitude','longitude']],y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=50, p=2,
                    weights='uniform')

In [270]:
len(np.unique(KNR.predict(x_train[['latitude','longitude']])))

872

In [271]:
KNR.score(x_train[['latitude','longitude']],y_train), KNR.score(x_test[['latitude','longitude']],y_test) #obviously can make this better. Do a grid search, just have to

(0.5339287638986265, 0.5499841603886684)

In [272]:
x_train['residuals'] = y_train - KNR.predict(x_train[['latitude','longitude']])

In [278]:
x_train.residuals

756      -2049.00
829      69275.94
624     194835.74
1690    -22893.26
1101     15627.00
          ...    
709     -29747.50
666      56364.90
733      -5036.00
2       -10364.60
167      34895.44
Name: residuals, Length: 1268, dtype: float64

In [274]:
from sklearn.linear_model import LinearRegression

house_base_reduced = x_train.loc[:,['GrLivArea','residuals','LotArea',
                      'OverallQual','OverallCond','YearBuilt','YearRemodAdd',
                      'TotalBsmtSF','Fireplaces','GarageArea',
                      'MoSold','YrSold']]
house_base_reduced = pd.get_dummies(house_base_reduced)

house_base_reduced['Age'] = house_base_reduced['YrSold'] - house_base_reduced['YearBuilt'] #think about combining into one
house_base_reduced['YrSinceRemod'] =  house_base_reduced['YrSold'] - house_base_reduced['YearRemodAdd']
house_base_reduced = house_base_reduced.drop(['YrSold','YearBuilt','YearRemodAdd'],axis = 1)

In [260]:
lm_residuals = LinearRegression()

In [275]:
house_base_reduced = house_base_reduced.dropna()

In [276]:
lm_residuals.fit(house_base_reduced.drop('residuals', axis =1), house_base_reduced['residuals'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [277]:
import statsmodels.api as sm
x = sm.add_constant(house_base_reduced.drop('residuals', axis =1))
# Notice that the dependent variable (y) comes before the independent variables (x)
# Use df.columns.values to get all the column names
model = sm.OLS(house_base_reduced['residuals'], x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              residuals   R-squared:                       0.498
Model:                            OLS   Adj. R-squared:                  0.494
Method:                 Least Squares   F-statistic:                     124.7
Date:                Wed, 26 Aug 2020   Prob (F-statistic):          3.50e-180
Time:                        19:05:15   Log-Likelihood:                -15102.
No. Observations:                1266   AIC:                         3.023e+04
Df Residuals:                    1255   BIC:                         3.028e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.937e+05   9937.538    -19.491   